In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

1.13.1
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()

In [4]:
x_valid,x_train =  x_train_all[:5000],x_train_all[5000:]
y_valid,y_train =  y_train_all[:5000],y_train_all[5000:]

In [5]:
# 对数据做标准化
# x = (x - u)/std (u:均值,std:方差)

from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()

# 需要一个二维矩阵 x_train是三维的，要转成二维矩阵，再转成三维
# [None,28,28] ---> [None,784]
x_train_sacled = scalar.fit_transform(
x_train.astype(np.float32).reshape(-1,1)
).reshape(-1,28*28)

x_valid_scaled = scalar.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

x_test_scaled = scalar.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

y_train = np.asarray(y_train,dtype=np.int64)
y_valid = np.asarray(y_valid,dtype=np.int64)
y_test = np.asarray(y_test,dtype=np.int64)


In [6]:
np.max(x_train_sacled),np.min(x_train_sacled)

(2.0231433, -0.8105136)

In [7]:
def make_dataset(images,labels,epochs,batch_size,shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [9]:
batch_size = 20
epochs = 10

images_placeholder = tf.placeholder(tf.float32,[None,28*28])
labels_placeholder = tf.placeholder(tf.int64,[None,])

dataset = make_dataset(images_placeholder,labels_placeholder,epochs=epochs,batch_size=batch_size)



In [11]:
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_sacled,y_train,epochs=epochs,batch_size=batch_size)

# 1.auto initialization
# 2.不能被重新初始化
dataset_iter = dataset.make_initializable_iterator()
x, y = dataset_iter.get_next()
with tf.Session() as sess:
    sess.run(dataset_iter.initializer,
            feed_dict = {
                images_placeholder:x_train_sacled,
                labels_placeholder:y_train
            }
            )
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)
    sess.run(
        dataset_iter.initializer,
        feed_dict = {
            images_placeholder:x_valid_scaled,
            labels_placeholder:y_valid,
        }
    )
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)


(20, 784)
(20,)
(20, 784)
(20,)
